# Connelly's Conjecture

Connelly asked where the singularity is in the inversive distances of a twisted octahedron with tangencies along all of the convex edges. 

This document will examine this numerically / experimentally. 

In [3]:
# Imports

# Algorithms:
from koebe.algorithms.incrementalConvexHull import incrConvexHull, orientationPointE3, randomConvexHullE3
from koebe.algorithms.hypPacker import *
from koebe.algorithms.sampling import surfaceSampling, boundarySampling
from koebe.algorithms.poissonDiskSampling import slowAmbientSurfaceSampling, slowAmbientBoundarySampling
from koebe.algorithms.cvt import weightedCVT, worldToImgPixelCoords
from koebe.algorithms.tutteEmbeddings import tutteEmbeddingE2

# Graphics:
from koebe.graphics.spherical2viewer import *
from koebe.graphics.euclidean2viewer import PoincareDiskViewer, makeStyle, E2Viewer

# Geometries:
from koebe.geometries.orientedProjective2 import DiskOP2
from koebe.geometries.euclidean3 import PointE3, VectorE3, SegmentE3
from koebe.geometries.euclidean2 import *
from koebe.geometries.commonOps import inner_product31

# Linear Algebra:
import numpy as np
from numpy.linalg import matrix_rank, norm
from scipy.linalg import null_space

# Image creation:
from PIL import Image, ImageDraw, ImageFilter

# Other:
import random

In [14]:
A = CircleE2(PointE2(-1, -math.sqrt(3)/2), radius = 1.0)
B = CircleE2(PointE2(1, -math.sqrt(3)/2), radius = 1.0)
C = CircleE2(PointE2(0, math.sqrt(3)/2), radius = 1.0)

viewer = E2Viewer(800, 800, scale = 1/ 400)

blueStyle = makeStyle(stroke="#00f", strokeWeight=0.5)
redStyle = makeStyle(stroke="#f00", strokeWeight=2.5)

viewer.addAll([A, B, C])
viewer.show()

<IPython.core.display.Javascript object>

E2Sketch(height=800, objects='[[{"type": "CircleE2", "center": [-1, -0.8660254037844386], "radius": 1.0, "styl…

In [18]:
from koebe.graphics.euclidean2viewer import UnitScaleE2Sketch, makeStyle
viewer = UnitScaleE2Sketch()
#viewer.addAll([v.data for v in poly.verts])poly = randomConvexHullE3(16)

#for _ in range(60):
poly = randomConvexHullE3(6)
poly.outerFace = poly.faces[0]
tutteGraph = tutteEmbeddingE2(poly)
segments = [SegmentE2(e.aDart.origin.data, e.aDart.twin.origin.data) for e in tutteGraph.edges]
viewer.addAll(segments)
viewer.show()

<IPython.core.display.Javascript object>

E2Sketch(height=500, objects='[[{"type": "SegmentE2", "endpoints": [[0.09999999999999985, -0.1732050807568876]…

In [45]:


dists = [(v.data - PointE3.O).normSq() for v in tutteGraph.verts]

closestToOriginIdx = dists.index(min(dists))
packing, _ = maximalPacking(
    tutteGraph, 
    num_passes=1000, 
    centerDartIdx = tutteGraph.darts.index(tutteGraph.verts[closestToOriginIdx].aDart)
)

pinVerts = [packing.verts[0], packing.verts[1], packing.verts[2]]
freeEdges = [packing.edges[0], packing.edges[5], packing.edges[8]]


# Run this to view the circle packing
viewer = E2Viewer(800, 800, scale = 1.0 / (4800*0.5-10))

blackStyle = makeStyle(stroke="#000", strokeWeight=3.0)
blueStyle = makeStyle(stroke="#00f", strokeWeight=0.5)
redStyle = makeStyle(stroke="#f00", strokeWeight=2.5)
grayStyle = makeStyle(stroke="#aaa", strokeWeight=1.0)

viewer.addAll([(v, grayStyle if v in pinVerts else blackStyle) 
               for v in packing.verts])

edgeSegs = [(SegmentE2(e.aDart.origin.data.toPoincareCircleE2().center, 
                      e.aDart.dest.data.toPoincareCircleE2().center), 
             redStyle if e in freeEdges else blueStyle)
            for e in packing.edges]

viewer.addAll(edgeSegs)
viewer.show()

<IPython.core.display.Javascript object>

E2Sketch(height=800, objects='[[{"type": "CircleE2", "center": [-0.4252622664499701, -0.30531249444264624], "r…

In [54]:
verts31 = [DiskOP2.fromCircleE2(v.data.toPoincareCircleE2()).toDiskS2() 
           for v in packing.verts]
verts31 += [DiskS2(0,0,0,0)] # Add in the origin.

# Labeling each vertex with its index in the vertex list is
# useful for building the rigidity matrix
for i in range(len(packing.verts)):
    packing.verts[i].name = i

edgeList = [(e.aDart.origin.name, e.aDart.dest.name) for e in packing.edges]
edgeList += [(vIdx, len(verts31) - 1) for vIdx in range(len(verts31) - 1)]

# Build the rigidity matrix. 
def edgeRow(edgeIdx, disks, edgeList):
    i, j = edgeList[edgeIdx]
    pi = tuple(disks[i])
    pj = tuple(disks[j])
    cij = [pj[0] - pi[0], pj[1] - pi[1], pj[2] - pi[2], pi[3] - pj[3]]
    cji = [pi[0] - pj[0], pi[1] - pj[1], pi[2] - pj[2], pj[3] - pi[3]]
    return np.array([
        cij if vIdx == i else
        cji if vIdx == j else
        [0,0,0,0]
        for vIdx in range(len(disks) - 1)
    ]).flatten()

def vertexIdxPin(vIdx, disks):
    r1 = np.array([[1, 0, 0, 0] if i == vIdx else [0, 0, 0, 0] 
                   for i in range(len(disks) - 1)]).flatten()
    r2 = np.array([[0, 1, 0, 0] if i == vIdx else [0, 0, 0, 0] 
                   for i in range(len(disks) - 1)]).flatten()
    r3 = np.array([[0, 0, 1, 0] if i == vIdx else [0, 0, 0, 0] 
                   for i in range(len(disks) - 1)]).flatten()
    r4 = np.array([[0, 0, 0, 1] if i == vIdx else [0, 0, 0, 0] 
                   for i in range(len(disks) - 1)]).flatten()
    return np.array([r1, r2, r3, r4])

def vertexPin(pinVertex, disks):
    vIdx = pinVertex.name
    return vertexIdxPin(vIdx, disks)

def rigidityMatrix(packing, disks, edgeList, pinVerts, freeEdges):
    edgeRows = np.array([
        edgeRow(eIdx, disks, edgeList) 
        for eIdx in range(len(edgeList)) 
        if eIdx >= len(packing.edges) or not packing.edges[eIdx] in freeEdges
    ])
    pinRows = []
    for pV in pinVerts:
        pinRows.append(vertexPin(pV, disks))
        
    pinRowsOrig = vertexIdxPin(len(disks) - 1, disks)
    return np.concatenate((edgeRows, pinRowsOrig, *pinRows))

In [74]:
R = rigidityMatrix(packing, verts31, edgeList, pinVerts, freeEdges)
ns = null_space(R)[:,0].reshape(len(verts31)-1,4)
ns.reshape(ns.shape[0]*ns.shape[1],1)


array([[-1.99064856e-18],
       [-3.52384329e-17],
       [ 3.31494443e-17],
       [-4.89071894e-18],
       [ 8.25455752e-17],
       [-4.53275752e-17],
       [ 2.07163436e-17],
       [-1.40806292e-17],
       [-5.27363697e-17],
       [-4.52585996e-17],
       [ 5.19070408e-17],
       [-1.68274882e-19],
       [ 2.33876034e-01],
       [-1.46470830e-02],
       [-7.95094587e-02],
       [-2.34912553e-01],
       [ 2.14202419e-01],
       [ 3.16726825e-02],
       [ 3.36513770e-03],
       [-2.16167854e-01],
       [ 6.12314958e-01],
       [ 1.00002997e-01],
       [-1.15685291e-01],
       [-6.25813717e-01]])

In [77]:
def newtonStepOneDisk(disk, ns, deltaT):
    a = disk.a + deltaT * ns[0]
    b = disk.b + deltaT * ns[1]
    c = disk.c + deltaT * ns[2]
    d = disk.d + deltaT * ns[3]
    inv31 = 1.0# / inner_product31(a, b, c, d, a, b, c, d)
    return DiskS2(a * inv31, b * inv31, c * inv31, d * inv31)

def newtonStep(packing, disks, edgeList, deltaT, prevNs, pinVerts, freeEdges, direction = 1):
    R = rigidityMatrix(packing, disks, edgeList, pinVerts, freeEdges)
    ns = null_space(R)[:,0].reshape(len(disks)-1, 4)
    if prevNs is not None:
        posDot = np.dot(np.transpose(ns.reshape(ns.shape[0]*ns.shape[1], 1)), prevNs.reshape(ns.shape[0]*ns.shape[1], 1))[0][0]
        negDot = np.dot(np.transpose((-ns).reshape(ns.shape[0]*ns.shape[1], 1)), prevNs.reshape(ns.shape[0]*ns.shape[1], 1))[0][0]
        if negDot > posDot:
            ns = -ns
    else:
        ns = ns * direction
    return [newtonStepOneDisk(disks[vIdx], ns[vIdx], deltaT) 
            for vIdx in range(len(disks) - 1)] + [disks[-1]], ns

def newtonAnimation(packing, initialDisks, edgeList, iterationCount, deltaT, pinVerts, freeEdges, direction = 1):
    animation = [initialDisks]
    prevNs = None
    for _ in range(iterationCount):
        nextFrame, prevNs = newtonStep(packing, animation[-1], edgeList, deltaT, prevNs, pinVerts, freeEdges, direction)
        animation.append(nextFrame)
    return animation

In [86]:
anim = newtonAnimation(packing, verts31, edgeList, 1600, 0.005, pinVerts, freeEdges, direction = -1)

In [88]:



# Run this to view the circle packing
viewer =  E2Viewer(800, 800, scale = 1.0 / 200)
viewer.addAll([v.sgProjectToOP2() for v in verts31[0:len(verts31)-1]])

viewer.pushAnimFrame()

for i in range(1600):#len(anim)):
    disksForward = [(d.sgProjectToOP2(), blueStyle) 
            for d in anim[i][:-1]]
    
    viewer.addAll([(anim[i][:-1][vIdx].sgProjectToOP2(), 
                    grayStyle if packing.verts[vIdx] in pinVerts else blackStyle) 
               for vIdx in range(len(packing.verts))])

    #edgeSegs = [(SegmentE2(e.aDart.origin.data.toPoincareCircleE2().center, 
    #                  e.aDart.dest.data.toPoincareCircleE2().center), 
    #         redStyle if e in freeEdges else blueStyle)
    #        for e in packing.edges]
    
    viewer.pushAnimFrame()
    viewer.addAll(disksForward)

# disksForward = [(d.sgProjectToOP2(), blueStyle) 
#             for d in anim[1][:-1]]
# viewer.addAll(disksForward)

viewer.show()
print(verts31[0])

<IPython.core.display.Javascript object>

E2Sketch(height=800, objects='[[{"type": "CircleE2", "center": [0.5022159585496939, 0.3217971262501374], "radi…

DiskS2(a=0.9529781004602507, b=0.6106249888852925, c=0.8505245328999402, d=-1.0470218995397493)


In [79]:
anim[0]

[DiskS2(a=0.9529781004602507, b=0.6106249888852925, c=0.8505245328999402, d=-1.0470218995397493),
 DiskS2(a=0.874436013735143, b=0.560298794690275, c=-0.9761964698999173, d=-1.125563986264857),
 DiskS2(a=0.9529781004602507, b=-1.0424020598187633, c=-0.0982486834589889, d=-1.0470218995397493),
 DiskS2(a=1.0022110590363262, b=-0.0, c=-0.0, d=-0.9977889409636737),
 DiskS2(a=0.99340567543067, b=0.16169427769467368, c=-0.09445510858773003, d=-1.00659432456933),
 DiskS2(a=0.9934056754312406, b=-0.0, c=-0.1872613333632371, d=-1.0065943245687594),
 DiskS2(a=0, b=0, c=0, d=0)]